[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BouleJaune/picselliaT/blob/master/Object_Detection_TF1.ipynb)



# Object-detection made easy
We will learn how to easily train an object detection model from a list of pre-trained models with the dataset you created on the picsell-IA platform.

# Prerequisite

In [ ]:
!git clone https://github.com/BouleJaune/picselliaT
%cd picselliaT/
!pip install picsellia
%tensorflow_version 1.x

# Imports

In [1]:
import sys
sys.path.append("slim")
from picsellia import Client
import picsell_utils
import tensorflow as tf

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Setting up your Picsell client


We need to connect with the Picsell platform to retrieve all of your project data. 

In [2]:
project_token = "05a91847-53a6-4b29-9014-d372d11836c7" # Token from the picsell-IA platform
api_token = "5890ad3e6a9701cd123ef91b69ad396de8c2d20a"
model_name = "ssd-inception-base"

clt = Client(api_token=api_token)
clt.checkout_project(project_token=project_token)
clt.checkout_network(model_name)

Welcome Xenio3, Glad to have you back


InvalidQueryError: The Network name you provided does not exists for this project

Once the client is initialized we can start the pre-processing of our data.

# Data pre-processing

Most of the pre-processing is done by the Tensorflow object-detection API. However we do need to put our data in a TFRecord format for the API to understand it and to create a protobuf config file describing all the parameters of our model, input pipeline, evaluation and training configuration ...

Hopefully everything is handled by the picsell-utils package. 

## Downloading data and generating the label map

First we need the annotations and images on our machine. We also need to generate a label map, mapping the labels names to a label ID that the Tensorflow object-detection API can comprehend. We simply run the commands below to achieve this.

In [3]:
clt.dl_annotations()
clt.generate_labelmap()
clt.train_test_split()
clt.dl_pictures()

Annotations pulled ...
Generating labelmap ...
Label_map.pbtxt crée @ mask-classification/ssd-inception-base/1/label_map.pbtxt
[==================================================]0 files were already on your machine
 113 PNG images have been downloaded to your machine
90 Images used for training, 23 Images used for validation
Repartition send ..


## TFRecord files creation

Tensorflow needs the data in a TFRecord format, a memory efficient format storing data as a sequence of binary records. We generate two .record files, one for the training and one for the evaluation.
We set the annotation type to rectangle so we can extract the bounding boxes and their label from the annotations and store them inside the records.

In [4]:
annotation_type = "rectangle" 
picsell_utils.create_record_files(label_path=clt.label_path, record_dir=clt.record_dir,
                         tfExample_generator=clt.tf_vars_generator, annotation_type=annotation_type)



Successfully created the TFRecords: mask-classification/ssd-inception-base/1/records/train.record
Successfully created the TFRecords: mask-classification/ssd-inception-base/1/records/eval.record


## Editing the configuration file

Every trainable object-detection model downloaded from the Picsell.ia hub is provided with the protobuf configuration file used to train it. We want to edit this file and set some variables specific to our project. 

Most of those variables are provided by the SDK, but it's still up to you to set up some of them. 
If you want to dwell inside the configuration file to have more control over your model, you can still open it up with after that and go change some settings like the learning rate decay strategy. But it's not the purpose of this guide.

In [5]:
#The number of steps you want your model to be trained on
nb_steps = 1000
#Your batch size, highly depending of your hardware and model architecture
#Some models will return errors while training with a really low batch size
batch_size = 16
#The learning rate used, it can be left to None to use the previous one. 
learning_rate = None

picsell_utils.edit_config(model_selected=clt.model_selected, 
            config_output_dir=clt.config_dir,
            record_dir=clt.record_dir, 
            label_map_path=clt.label_path, 
            num_steps=nb_steps,
            batch_size=batch_size, 
            learning_rate=learning_rate,
            annotation_type=annotation_type,
            eval_number = len(clt.eval_list))



INFO:tensorflow:Writing pipeline config file to mask-classification/ssd-inception-base/1/config/pipeline.config


## Training

Now that the input pipeline is built we can finally launch the training. To do this we can use the wrapper function train from picsell_utils , we simply specify where we will save the checkpoint and where the configuration file is located.

In [6]:
picsell_utils.train(ckpt_dir=clt.checkpoint_dir, 
                     conf_dir=clt.config_dir)





Instructions for updating:
Please switch to tf.train.create_global_step




Instructions for updating:
Use `tf.data.experimental.parallel_interleave(...)`.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.map()
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.






Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1/checkpoint/origin/model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1/checkpoint/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:global step 1: loss = 17.9094 (19.995 sec/step)
INFO:tensorflow:global step 2: loss = 16.9508 (0.498 sec/step)
INFO:tensorflow:global step 3: los

INFO:tensorflow:global step 86: loss = 6.3487 (0.703 sec/step)
INFO:tensorflow:Recording summary at step 86.
INFO:tensorflow:global step 87: loss = 5.0957 (0.699 sec/step)
INFO:tensorflow:global step 88: loss = 3.3890 (0.465 sec/step)
INFO:tensorflow:global step 89: loss = 5.3825 (0.458 sec/step)
INFO:tensorflow:global step 90: loss = 4.6677 (0.475 sec/step)
INFO:tensorflow:global step 91: loss = 4.5312 (0.465 sec/step)
INFO:tensorflow:global step 92: loss = 5.0569 (0.465 sec/step)
INFO:tensorflow:global step 93: loss = 4.9862 (0.454 sec/step)
INFO:tensorflow:global_step/sec: 2.13569
INFO:tensorflow:global step 94: loss = 5.3966 (0.474 sec/step)
INFO:tensorflow:global step 95: loss = 4.8116 (0.460 sec/step)
INFO:tensorflow:global step 96: loss = 4.5868 (0.458 sec/step)
INFO:tensorflow:global step 97: loss = 6.0578 (0.457 sec/step)
INFO:tensorflow:global step 98: loss = 5.0496 (0.468 sec/step)
INFO:tensorflow:global step 99: loss = 5.2516 (0.453 sec/step)
INFO:tensorflow:global step 100

INFO:tensorflow:global step 212: loss = 4.2074 (0.472 sec/step)
INFO:tensorflow:global step 213: loss = 4.8438 (0.605 sec/step)
INFO:tensorflow:Recording summary at step 213.
INFO:tensorflow:global step 214: loss = 3.0680 (0.707 sec/step)
INFO:tensorflow:global step 215: loss = 4.9631 (0.469 sec/step)
INFO:tensorflow:global step 216: loss = 3.9019 (0.464 sec/step)
INFO:tensorflow:global step 217: loss = 3.2373 (0.457 sec/step)
INFO:tensorflow:global step 218: loss = 4.2297 (0.459 sec/step)
INFO:tensorflow:global step 219: loss = 3.8388 (0.470 sec/step)
INFO:tensorflow:global step 220: loss = 2.9336 (0.476 sec/step)
INFO:tensorflow:global_step/sec: 2.13574
INFO:tensorflow:global step 221: loss = 4.7241 (0.484 sec/step)
INFO:tensorflow:global step 222: loss = 4.2385 (0.457 sec/step)
INFO:tensorflow:global step 223: loss = 4.0768 (0.462 sec/step)
INFO:tensorflow:global step 224: loss = 4.4188 (0.454 sec/step)
INFO:tensorflow:global step 225: loss = 3.8068 (0.458 sec/step)
INFO:tensorflow:

INFO:tensorflow:global step 338: loss = 4.0225 (0.725 sec/step)
INFO:tensorflow:Recording summary at step 338.
INFO:tensorflow:global step 339: loss = 2.6609 (0.489 sec/step)
INFO:tensorflow:global step 340: loss = 2.7996 (0.459 sec/step)
INFO:tensorflow:global step 341: loss = 3.2887 (0.455 sec/step)
INFO:tensorflow:global step 342: loss = 4.5026 (0.456 sec/step)
INFO:tensorflow:global step 343: loss = 2.8404 (0.469 sec/step)
INFO:tensorflow:global step 344: loss = 4.5667 (0.466 sec/step)
INFO:tensorflow:global_step/sec: 2.06962
INFO:tensorflow:global step 345: loss = 3.4594 (0.470 sec/step)
INFO:tensorflow:global step 346: loss = 2.9454 (0.461 sec/step)
INFO:tensorflow:global step 347: loss = 2.3753 (0.480 sec/step)
INFO:tensorflow:global step 348: loss = 4.0999 (0.462 sec/step)
INFO:tensorflow:global step 349: loss = 4.0784 (0.460 sec/step)
INFO:tensorflow:global step 350: loss = 4.0461 (0.459 sec/step)
INFO:tensorflow:global step 351: loss = 2.3226 (0.469 sec/step)
INFO:tensorflow:

...

INFO:tensorflow:global step 589: loss = 4.2389 (0.503 sec/step)
INFO:tensorflow:global step 590: loss = 1.7654 (0.485 sec/step)
INFO:tensorflow:global step 591: loss = 1.9244 (0.487 sec/step)
INFO:tensorflow:global step 592: loss = 2.3706 (0.469 sec/step)
INFO:tensorflow:global step 593: loss = 3.2308 (0.470 sec/step)
INFO:tensorflow:global step 594: loss = 3.1373 (0.472 sec/step)
INFO:tensorflow:global step 595: loss = 3.0130 (0.466 sec/step)
INFO:tensorflow:global step 596: loss = 3.1031 (0.485 sec/step)
INFO:tensorflow:global step 597: loss = 2.4714 (0.483 sec/step)
INFO:tensorflow:global step 598: loss = 2.3869 (0.472 sec/step)
INFO:tensorflow:global step 599: loss = 2.7383 (0.472 sec/step)
INFO:tensorflow:global step 600: loss = 2.8196 (0.468 sec/step)
INFO:tensorflow:global step 601: loss = 4.1617 (0.471 sec/step)
INFO:tensorflow:global step 602: loss = 2.8166 (0.507 sec/step)
INFO:tensorflow:global step 603: loss = 4.0632 (0.490 sec/step)
INFO:tensorflow:global step 604: loss = 

INFO:tensorflow:global step 716: loss = 2.7863 (0.481 sec/step)
INFO:tensorflow:global step 717: loss = 2.4377 (0.475 sec/step)
INFO:tensorflow:global step 718: loss = 2.1695 (0.506 sec/step)
INFO:tensorflow:global step 719: loss = 3.0293 (0.473 sec/step)
INFO:tensorflow:global step 720: loss = 2.0162 (0.466 sec/step)
INFO:tensorflow:global step 721: loss = 2.7067 (0.475 sec/step)
INFO:tensorflow:global step 722: loss = 2.7878 (0.473 sec/step)
INFO:tensorflow:global step 723: loss = 3.4682 (0.478 sec/step)
INFO:tensorflow:global step 724: loss = 2.3045 (0.461 sec/step)
INFO:tensorflow:global step 725: loss = 2.1727 (0.466 sec/step)
INFO:tensorflow:global step 726: loss = 2.4350 (0.470 sec/step)
INFO:tensorflow:global step 727: loss = 2.5173 (0.484 sec/step)
INFO:tensorflow:global step 728: loss = 2.6747 (0.470 sec/step)
INFO:tensorflow:global step 729: loss = 3.4426 (0.486 sec/step)
INFO:tensorflow:global step 730: loss = 2.7025 (0.492 sec/step)
INFO:tensorflow:global step 731: loss = 

INFO:tensorflow:global step 843: loss = 2.4337 (0.503 sec/step)
INFO:tensorflow:global step 844: loss = 2.1998 (0.476 sec/step)
INFO:tensorflow:global step 845: loss = 3.2138 (0.476 sec/step)
INFO:tensorflow:global step 846: loss = 2.3534 (0.497 sec/step)
INFO:tensorflow:global step 847: loss = 2.5561 (0.478 sec/step)
INFO:tensorflow:global step 848: loss = 3.4175 (0.472 sec/step)
INFO:tensorflow:global step 849: loss = 4.3649 (0.483 sec/step)
INFO:tensorflow:global step 850: loss = 3.4348 (0.463 sec/step)
INFO:tensorflow:global step 851: loss = 2.7898 (0.471 sec/step)
INFO:tensorflow:global step 852: loss = 2.0154 (0.485 sec/step)
INFO:tensorflow:global step 853: loss = 3.1202 (0.504 sec/step)
INFO:tensorflow:global step 854: loss = 2.2979 (0.478 sec/step)
INFO:tensorflow:global step 855: loss = 2.0225 (0.498 sec/step)
INFO:tensorflow:global step 856: loss = 2.7835 (0.468 sec/step)
INFO:tensorflow:global step 857: loss = 2.5660 (0.464 sec/step)
INFO:tensorflow:global step 858: loss = 

INFO:tensorflow:global step 970: loss = 3.6793 (0.480 sec/step)
INFO:tensorflow:global step 971: loss = 3.4115 (0.475 sec/step)
INFO:tensorflow:global step 972: loss = 2.0142 (0.464 sec/step)
INFO:tensorflow:global step 973: loss = 2.8946 (0.467 sec/step)
INFO:tensorflow:global step 974: loss = 2.2328 (0.470 sec/step)
INFO:tensorflow:global step 975: loss = 2.2433 (0.483 sec/step)
INFO:tensorflow:global step 976: loss = 2.5286 (0.461 sec/step)
INFO:tensorflow:global step 977: loss = 3.3229 (0.467 sec/step)
INFO:tensorflow:global step 978: loss = 2.5603 (0.471 sec/step)
INFO:tensorflow:global step 979: loss = 2.2924 (0.467 sec/step)
INFO:tensorflow:global step 980: loss = 2.3105 (0.467 sec/step)
INFO:tensorflow:global step 981: loss = 2.1580 (0.474 sec/step)
INFO:tensorflow:global step 982: loss = 2.4990 (0.461 sec/step)
INFO:tensorflow:global step 983: loss = 2.5092 (0.470 sec/step)
INFO:tensorflow:global step 984: loss = 2.3902 (0.482 sec/step)
INFO:tensorflow:global step 985: loss = 

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/summary/writer/writer.py:386: UserWarning: Attempting to use a closed FileWriter. The operation will be a noop unless the FileWriter is explicitly reopened.
  warnings.warn("Attempting to use a closed FileWriter. "


Once the training is done we want to send the training logs to the Picsell.ia platform.

In [7]:
dict_log = picsell_utils.tfevents_to_dict(path=clt.checkpoint_dir)
clt.send_logs(dict_log)

Training logs have been sent to Picsell.ia Platform...
You can now inspect and showcase results on the platform.


## Evaluating

Next is the evaluation phase, we can launch the evaluation with the evaluate method of the picsell_utils module.

In [8]:
metrics = picsell_utils.evaluate(clt.metrics_dir, clt.config_dir, clt.checkpoint_dir)
clt.send_metrics(metrics)

INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Starting evaluation at 2020-05-20-14:13:37
INFO:tensorflow:Restoring parameters from 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1/checkpoint/model.ckpt-1000
INFO:tensorflow:Running eval batches done.


INFO:tensorflow:Running eval batches done.


INFO:tensorflow:# success: 19


INFO:tensorflow:# success: 19


INFO:tensorflow:# skipped: 4


...

INFO:tensorflow:Metrics written to tf summary.


INFO:tensorflow:Metrics written to tf summary.


INFO:tensorflow:Finished evaluation!


INFO:tensorflow:Finished evaluation!


Evaluation metrics have been sent to Picsell.ia Platform...
You can now inspect and showcase results on the platform.


You can send the checkpoints to the Picsell.ia platform now.

In [ ]:
clt.send_checkpoints()

# Exporting and infering

Now that the training is done and that we checked the performance of our model through the metrics returned by the evaluation we may want to export and use this model.

## Exporting the model

To export our model as a saved_model.pb we use the export_infer_graph function while specifying the right paths. We can send it to the platform to be used on the playground for live inference.

In [9]:
picsell_utils.export_infer_graph(ckpt_dir=clt.checkpoint_dir, 
                       exported_model_dir=clt.exported_model_dir, 
                       pipeline_config_path=clt.config_dir)

clt.send_model()

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:SavedModel written to: 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1exported_model/saved_model/saved_model.pb


INFO:tensorflow:SavedModel written to: 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1exported_model/saved_model/saved_model.pb


INFO:tensorflow:Writing pipeline config file to 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1exported_model/pipeline.config


INFO:tensorflow:Writing pipeline config file to 05a91847-53a6-4b29-9014-d372d11836c7/d21c776e-7595-42c1-9846-54713135b78e/1exported_model/pipeline.config


IndexError: tuple index out of range

## Inference

Lastly we want to use the model on some images of our evaluation set so we can send the results to the platform and see them on the dashboard.

In [ ]:
picsell_utils.infer(clt.eval_list, exported_model_dir=clt.exported_model_dir, 
      label_map_path=clt.label_path, results_dir=clt.results_dir)
clt.send_examples()